### Import Libraries

In [11]:
import random
import numpy as np
import igraph 
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import csv
import pandas as pd
import networkx as nx
import math
import time

nltk.download('punkt') # for tokenization
nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()


[nltk_data] Downloading package punkt to /Users/taneja/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/taneja/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Load Data

In [2]:
with open("training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]
# [['9510123', '9502114', '1'],
#  ['9707075', '9604178', '1'],

with open("testing_set.txt", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)

testing_set = [element[0].split(" ") for element in testing_set]

with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

IDs = [(element[0]) for element in node_info]
print('Min and Max and Length of IDs: ', min(IDs),max(IDs),len(IDs))

# compute TFIDF vector of each paper
corpus = [element[5] for element in node_info]
vectorizer = TfidfVectorizer(stop_words="english")
features_TFIDF = vectorizer.fit_transform(corpus)

print('Length of corpus',len(corpus))
print("Shape of the train TF-IDF Matrix: {}".format(features_TFIDF.shape))

Min and Max and Length of IDs:  10001 9912293 27770
Length of corpus 27770
Shape of the train TF-IDF Matrix: (27770, 25043)


#### Cross-Checks On Data

In [3]:
all_train_Ids = []
for x in training_set:
    all_train_Ids.append(x[0])
    all_train_Ids.append(x[1])
print('Count and Distinct count of all Train Ids:',len(all_train_Ids),len(set(all_train_Ids)))
    
    
all_test_Ids = []
for x in testing_set:
    all_test_Ids.append(x[0])
    all_test_Ids.append(x[1])
print('Count and Distinct count of all Test Ids:',len(all_test_Ids),len(set(all_test_Ids)))
    
# Referring to slide 11 in link prediction pdf   
print('No.of train Ids that are not part of nodes:',len(set(all_train_Ids)-set(IDs)))
print('No.of test Ids that are not part of nodes:',len(set(all_test_Ids)-set(IDs)))

print('No.of nodes that are not part of train Ids:',len(set(IDs)-set(all_train_Ids)))
print('No.of nodes that are not part of test Ids:',len(set(IDs)-set(all_test_Ids)))

Count and Distinct count of all Train Ids: 1231024 27770
Count and Distinct count of all Test Ids: 65296 23402
No.of train Ids that are not part of nodes: 0
No.of test Ids that are not part of nodes: 0
No.of nodes that are not part of train Ids: 0
No.of nodes that are not part of test Ids: 4368


#### Create Graphs in NetworkX and iGraph

In [4]:
edges = [(element[0],element[1]) for element in training_set if element[2]=="1"]

# some nodes may not be connected to any other node hence the need to create nodes, not just from the edge list
nodes = IDs

## iGraph
g = igraph.Graph(directed=True)
g.add_vertices(nodes)
g.add_edges(edges)
g.summary()
print('iGraph nodes and edges:',g.vcount(),g.ecount())

# NetworkX
G=nx.Graph()
G.add_edges_from(edges)
print('NetworkX nodes and edges:',G.number_of_nodes(),G.number_of_edges())

iGraph nodes and edges: 27770 335130
NetworkX nodes and edges: 27684 334690


#### Identify All Graph Features

In [5]:
source = '1001'
target = '1002'

index_source = IDs.index(source)
index_target = IDs.index(target)

# n  - Node
# st - Source & Target

def n_in_out_degree_prod(node_name):
    prod = g.degree([node_name],type="in")[0]*g.degree([node_name],type="out")[0]
    return(prod)
print('n_in_out_degree_prod:',n_in_out_degree_prod(source))


def st_in_degree_product(source,target):
    prod = g.degree(source,type="in") * g.degree(target,type="in")
    return(prod)
print('st_in_degree_product:',st_in_degree_product(source,target))


def st_out_degree_product(source,target):
    prod = g.degree(source,type="out") * g.degree(target,type="out")
    return(prod)
print('st_out_degree_product:',st_out_degree_product(source,target))

def st_in_out_degree_product(source,target):
    prod = g.degree(source) * g.degree(target)
    return(prod)
print('st_in_out_degree_product:',st_in_out_degree_product(source,target))

def st_common_neighbors(source,target):
    count = len(set(g.neighbors(source)).intersection(g.neighbors(target)))
    return(count)
print('st_common_neighbors:',st_common_neighbors(source,target))

def st_jaccard_similarity(source,target):
    nr = len(set(g.neighbors(source)).intersection(set(g.neighbors(target))))
    dr = float(len(set(g.neighbors(source)).union(set(g.neighbors(target)))))
    return(nr/dr)
print('st_jaccard_similarity:',st_jaccard_similarity(source,target))

def st_adamic_adar(source,target):
    ans = sum([1.0/math.log(g.degree(v)) for v in set(g.neighbors(source)).intersection(set(g.neighbors(target)))])
    return(ans)
print('st_adamic_adar:',st_adamic_adar(source,target))

def st_friendtns(source,target):
    ans = round((1.0/(g.degree(source) + g.degree(target) - 1.0)),3)
    return(ans)
print('st_friendtns:',st_friendtns(source,target))
 
def katz(g):
    katzDict = {}  # build a special dict that is like an adjacency list
    adjList = g.get_adjlist()

    for i, l in enumerate(adjList):
        katzDict[i] = l
    return katzDict
        
def st_katz_similarity(katzDict,i,j):
    l = 1
    maxl = 5
    neighbors = katzDict[i]
    score = 0
    beta = 0.005 

    while l <= maxl:
        numberOfPaths = neighbors.count(j)
        if numberOfPaths > 0:
            score += (beta**l)*numberOfPaths

        neighborsForNextLoop = []
        for k in neighbors:
            neighborsForNextLoop += katzDict[k]
        neighbors = neighborsForNextLoop
        l += 1

    return score

katzDict = katz(g)
print('st_katz_similarity',st_katz_similarity(katzDict,index_source,index_target))

def st_count_shortest_paths_dijkstra(source,target):
    ans = g.shortest_paths_dijkstra(source, target)[0][0]
    return(ans)
print('st_count_shortest_paths_dijkstra:',st_count_shortest_paths_dijkstra(source,target))

def st_count_nodes_in_paths(source,target):
    s=set(g.subcomponent(source, mode="out"))
    t=set(g.subcomponent(target, mode="in"))
    return(len(s.intersection(t)))
print('st_count_nodes_in_paths:',st_count_nodes_in_paths(source,target))

def st_closeness(source,target):
    ans = g.closeness(vertices=source)+g.closeness(vertices=target)
    return(ans)
print('st_closeness:',st_closeness(source,target))


/Users/taneja/Anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: type=... keyword argument is deprecated since igraph 0.6, use mode=... instead
  # This is added back by InteractiveShellApp.init_path()
/Users/taneja/Anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: type=... keyword argument is deprecated since igraph 0.6, use mode=... instead
/Users/taneja/Anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: type=... keyword argument is deprecated since igraph 0.6, use mode=... instead


n_in_out_degree_prod: 800
st_in_degree_product: 420
st_out_degree_product: 2800
st_in_out_degree_product: 6930
st_common_neighbors: 0
st_jaccard_similarity: 0.0
st_adamic_adar: 0
st_friendtns: 0.006
st_katz_similarity 0
st_count_shortest_paths_dijkstra: 7
st_count_nodes_in_paths: 7229
st_closeness: 0.004094187311730543


/Users/taneja/Anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:96: RuntimeWarning: closeness centrality is not well-defined for disconnected graphs at /Users/travis/build/igraph/python-igraph/vendor/source/igraph/src/centrality.c:2856


#### Build Features Of Train Data

In [ ]:
# randomly select 5% of training set
to_keep = random.sample(range(len(training_set)), k=int(round(len(training_set)*0.05)))
training_set_reduced = [training_set[i] for i in to_keep]
# training_set_reduced = training_set

overlap_title = []                         # number of overlapping words in title
temp_diff = []                             # temporal distance between the papers
comm_auth = []                             # number of common authors
ln_in_out_degree_prod_source = []          # 
ln_in_out_degree_prod_target = []          # 
lst_in_degree_product = []                 # 
lst_out_degree_product = []                # 
lst_in_out_degree_product = []             # 
lst_common_neighbors = []                  # 
lst_jaccard_similarity = []                # 
lst_adamic_adar = []                       # 
lst_friendtns = []                         # 
lst_katz_similarity = []                   # 
lst_count_shortest_paths_dijkstra = []     # 
lst_count_nodes_in_paths = []              # 
lst_closeness = []                         # 

counter = 0
start_time = time.time()

for i in range(len(training_set_reduced)):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    # Metadata Features
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    source_title = source_info[2].lower().split(" ")
    source_title = [token for token in source_title if token not in stpwds]
    source_title = [stemmer.stem(token) for token in source_title]
    
    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]
    
    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",")
    
    overlap_title.append(len(set(source_title).intersection(set(target_title))))
    temp_diff.append(int(source_info[1]) - int(target_info[1]))
    comm_auth.append(len(set(source_auth).intersection(set(target_auth))))
   
    # Graph Features
    ln_in_out_degree_prod_source.append(n_in_out_degree_prod(source))
    ln_in_out_degree_prod_target.append(n_in_out_degree_prod(target))
    lst_in_degree_product.append(st_in_degree_product(source,target))
    lst_out_degree_product.append(st_out_degree_product(source,target))
    lst_in_out_degree_product.append(st_in_out_degree_product(source,target))
    lst_common_neighbors.append(st_common_neighbors(source,target))
    lst_jaccard_similarity.append(st_jaccard_similarity(source,target))
    lst_adamic_adar.append(st_adamic_adar(source,target))
#     lst_friendtns.append(st_friendtns(source,target))
#     lst_katz_similarity.append(st_katz_similarity(katzDict,index_source,index_target))
    lst_count_shortest_paths_dijkstra.append(st_count_shortest_paths_dijkstra(source,target))
    lst_count_nodes_in_paths.append(st_count_nodes_in_paths(source,target))
    lst_closeness.append(st_closeness(source,target))

    counter += 1
    if counter % 1000 == True:
#         print(counter, "training examples processsed")
            print(f'Training time for {counter} rows is {int((time.time()-start_time)/60)} mins')

# convert list of lists into array
training_features = np.array([
                            overlap_title, 
                            temp_diff, 
                            comm_auth,
                            ln_in_out_degree_prod_source,
                            ln_in_out_degree_prod_target,
                            lst_in_degree_product,
                            lst_out_degree_product,
                            lst_in_out_degree_product,
                            lst_common_neighbors,
                            lst_jaccard_similarity,
                            lst_adamic_adar,
#                             lst_friendtns,
#                             lst_katz_similarity,
                            lst_count_shortest_paths_dijkstra,
                            lst_count_nodes_in_paths,
                            lst_closeness]).T

print('Total Train time in minutes',int((time.time()-start_time)/60))

# scale
training_features = preprocessing.scale(training_features)

# convert labels into integers then into column array
labels = [int(element[2]) for element in training_set_reduced]
labels = list(labels)
labels_array = np.array(labels)


/Users/taneja/Anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: type=... keyword argument is deprecated since igraph 0.6, use mode=... instead
  # This is added back by InteractiveShellApp.init_path()
/Users/taneja/Anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: type=... keyword argument is deprecated since igraph 0.6, use mode=... instead
/Users/taneja/Anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: type=... keyword argument is deprecated since igraph 0.6, use mode=... instead
/Users/taneja/Anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:96: RuntimeWarning: closeness centrality is not well-defined for disconnected graphs at /Users/travis/build/igraph/python-igraph/vendor/source/igraph/src/centrality.c:2856


Training time for 1 rows is 0 mins
Training time for 1001 rows is 1 mins
Training time for 2001 rows is 2 mins
Training time for 3001 rows is 3 mins
Training time for 4001 rows is 4 mins
Training time for 5001 rows is 5 mins
Training time for 6001 rows is 6 mins
Training time for 7001 rows is 8 mins
Training time for 8001 rows is 9 mins
Training time for 9001 rows is 10 mins
Training time for 10001 rows is 11 mins
Training time for 11001 rows is 12 mins
Training time for 12001 rows is 13 mins
Training time for 13001 rows is 15 mins
Training time for 14001 rows is 16 mins
Training time for 15001 rows is 17 mins
Training time for 16001 rows is 18 mins
Training time for 17001 rows is 19 mins
Training time for 18001 rows is 20 mins
Training time for 19001 rows is 21 mins
Training time for 20001 rows is 23 mins
Training time for 21001 rows is 24 mins


#### Build Features Of Test Data

In [ ]:
overlap_title = []                         # number of overlapping words in title
temp_diff = []                             # temporal distance between the papers
comm_auth = []                             # number of common authors
ln_in_out_degree_prod_source = []          # 
ln_in_out_degree_prod_target = []          # 
lst_in_degree_product = []                 # 
lst_out_degree_product = []                # 
lst_in_out_degree_product = []             # 
lst_common_neighbors = []                  # 
lst_jaccard_similarity = []                # 
lst_adamic_adar = []                       # 
lst_friendtns = []                         # 
lst_katz_similarity = []                   # 
lst_count_shortest_paths_dijkstra = []     # 
lst_count_nodes_in_paths = []              # 
lst_closeness = []                         # 

counter = 0
start_time = time.time()

for i in range(len(testing_set)):
    source = testing_set[i][0]
    target = testing_set[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)

    # Metadata Features    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    source_title = source_info[2].lower().split(" ")
    source_title = [token for token in source_title if token not in stpwds]
    source_title = [stemmer.stem(token) for token in source_title]
    
    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]
    
    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",")
    
    overlap_title_test.append(len(set(source_title).intersection(set(target_title))))
    temp_diff_test.append(int(source_info[1]) - int(target_info[1]))
    comm_auth_test.append(len(set(source_auth).intersection(set(target_auth))))

    # Graph Features
    ln_in_out_degree_prod_source.append(n_in_out_degree_prod(source))
    ln_in_out_degree_prod_target.append(n_in_out_degree_prod(target))
    lst_in_degree_product.append(st_in_degree_product(source,target))
    lst_out_degree_product.append(st_out_degree_product(source,target))
    lst_in_out_degree_product.append(st_in_out_degree_product(source,target))
    lst_common_neighbors.append(st_common_neighbors(source,target))
    lst_jaccard_similarity.append(st_jaccard_similarity(source,target))
    lst_adamic_adar.append(st_adamic_adar(source,target))
#     lst_friendtns.append(st_friendtns(source,target))
#     lst_katz_similarity.append(st_katz_similarity(katzDict,index_source,index_target))
    lst_count_shortest_paths_dijkstra.append(st_count_shortest_paths_dijkstra(source,target))
    lst_count_nodes_in_paths.append(st_count_nodes_in_paths(source,target))
    lst_closeness.append(st_closeness(source,target))
    
    counter += 1
    if counter % 1000 == True:
#         print(counter, "testing examples processsed")
        print(f'Testing time for {counter} rows is {int((time.time()-start_time)/60)} mins')
        
# convert list of lists into array
testing_features = np.array([
                            overlap_title, 
                            temp_diff, 
                            comm_auth,
                            ln_in_out_degree_prod_source,
                            ln_in_out_degree_prod_target,
                            lst_in_degree_product,
                            lst_out_degree_product,
                            lst_in_out_degree_product,
                            lst_common_neighbors,
                            lst_jaccard_similarity,
                            lst_adamic_adar,
#                             lst_friendtns,
#                             lst_katz_similarity,
                            lst_count_shortest_paths_dijkstra,
                            lst_count_nodes_in_paths,
                            lst_closeness]).T

print('Total Train time in minutes',int((time.time()-start_time)/60))

# scale
testing_features = preprocessing.scale(testing_features)

### Build Models

In [ ]:
# SVM
classifier = svm.LinearSVC()
classifier.fit(training_features, labels_array)
predictions_SVM = list(classifier.predict(testing_features))

df_out = pd.DataFrame({'id':range(len(testing_set)),'category':predictions_SVM})
print(df_out.head())
df_out.to_csv('sub4.csv',index=False)

__Experiments__
- Sub1: 100% data | 0.53548 | wrong code in testing part <br>
_Corrected code in the testing part_
- Sub2: 5% data   | 0.71251 |
- Sub3: 100% data | 0.71251 | doubtful that its the same as Sub2 <br>
_Until now only 3 features are used_

***

__Next Steps__
- Add tf-idf cosine similarity between source and target as extra feature
- Check if Gensim is sensible to do instead of tf-idf
- Apply keras neural network with all features including tf-idf
